In [46]:
import torch
import torch.nn as nn
import pandas as pd
from torchvision import transforms
from PIL import Image
import os

numeric_to_label = []
class LeafDataset:
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        self.label_to_numeric = {label: idx for idx, label in enumerate(sorted(set(self.data['labels'])))}
        self.data['labels'] = self.data['labels'].map(self.label_to_numeric)
        
        numeric_to_label = self.label_to_numeric

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)  
        self.fc2 = nn.Linear(128, 12)  
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 56 * 56)  
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


cnn_model = CNN()

checkpoint = torch.load('/kaggle/input/plant_path_final/pytorch/final/1/plant_path_trained3apr.pth')

cnn_model.load_state_dict(checkpoint['model_state_dict'])

test_dataset = LeafDataset(csv_file='plant_path_submit.csv', root_dir='test_images', transform=transform)
print(train_dataset)

def predict_image_label(image):
    with torch.no_grad():
        output = cnn_model(image.unsqueeze(0)[0])
        print(output)
        _, predicted = torch.max(output, 1)
        numeric_label = predicted.item()
        return numeric_label




In [47]:
data = pd.read_csv("train.csv")
label_to_numeric = {labels: idx for idx, labels in enumerate(sorted(set(data['labels'])))}

print(label_to_numeric)
        



{'complex': 0, 'frog_eye_leaf_spot': 1, 'frog_eye_leaf_spot complex': 2, 'healthy': 3, 'powdery_mildew': 4, 'powdery_mildew complex': 5, 'rust': 6, 'rust complex': 7, 'rust frog_eye_leaf_spot': 8, 'scab': 9, 'scab frog_eye_leaf_spot': 10, 'scab frog_eye_leaf_spot complex': 11}


In [48]:
pred = []
for i in range(len(test_dataset)):
    image, label = test_dataset[i]
    
    predicted_label_numeric = predict_image_label(image)
    print(predicted_label_numeric)
    pred.append(predicted_label_numeric)

tensor([[ 0.0148,  1.1053, -2.8359,  4.2099, -2.0991, -3.9525,  5.8089,  0.3069,
          1.0018,  2.4787, -0.1662, -3.8287]])
6
tensor([[ 3.1203,  1.6698,  1.5924,  0.0472,  1.7323,  0.0831, -0.1354, -1.1134,
         -1.1551,  1.9359,  0.9040,  0.8466]])
0
tensor([[ 1.1330,  2.1334, -0.9146,  2.4205,  2.9996, -1.7709, -1.5686, -3.4729,
         -3.3620,  1.5697,  1.5258, -0.9668]])
4


In [49]:
label_to_name = {v: k for k, v in label_to_numeric.items()}

In [50]:
images = pd.read_csv("pathology.csv")["image"]
print(images)

0    85f8cb619c66b863.jpg
1    ad8770db05586b59.jpg
2    c7b03e718489f3ca.jpg
Name: image, dtype: object


In [51]:
predicted_labels = []
for i in pred:
    predicted_labels.append(label_to_name[i])
    
df = pd.DataFrame({'image': images, 'labels': predicted_labels})

df.to_csv('predictions.csv', index=False)